# References
- ["Exploring UPnP with Python" - Electric Monk](https://www.electricmonk.nl/log/2016/07/05/exploring-upnp-with-python/)
  - A lot of the SSDP code is a direct copy from this reference
- [upnpclient](https://github.com/flyte/upnpclient/blob/develop/upnpclient/soap.py)
  - A lot of the SOAP request code is a combination of this reference and the first reference

# Motivation
I just wanted to play around with UPNP device discovery. I'm sure you can find better code out there, but I do like the way I condensed down the Action and Argument definitions so it is really easy to see what is available.

# Terms
- **UPNP**: Universal Plug & Play
- **SSDP**: Simple Service Discovery Protocol
- **SCPD**: Service Control Point Definition
- **SOAP**: Simple Object Access Protocol

# SSDP
###           HTTPU M-Search UDP Broadcast  To UPNP Devices  ----->
### <-----  HTTPU UDP Replies with Location Header

In [140]:
import socket
from lxml import etree
from io import StringIO, BytesIO
import requests
from urllib.parse import urljoin
import xml.dom.minidom

class UPNP_SSDP:
    UPNP_BROADCAST_PORT = 1900
    UPNP_BROADCAST_IP = '239.255.255.250'
    UPNP_BRADCAST_MSG = \
            'M-SEARCH * HTTP/1.1\r\n' \
            'HOST:239.255.255.250:1900\r\n' \
            'ST:upnp:rootdevice\r\n' \
            'MX:2\r\n' \
            'MAN:"ssdp:discover"\r\n' \
            '\r\n'
    
    def __init__(self, print_broadcast=False):
        self.print_broadcast = print_broadcast
        self.broadcast()
        
    def broadcast(self, socket_timeout=2):
        # Set up UDP socket
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
        s.settimeout(socket_timeout)
        s.sendto(str.encode(self.UPNP_BRADCAST_MSG), (self.UPNP_BROADCAST_IP, self.UPNP_BROADCAST_PORT))
        
        self.servers = {}
        try:
            while True:
                data, addr = s.recvfrom(4096)
                server, property_dict = self.get_response_dict(data.decode())

                server += '_0'
                if server in self.servers.keys():
                    cnt = int(server.split('_')[-1]) + 1
                    server = server[:-len(str(cnt-1))] + str(cnt)        

                self.servers[server] = property_dict
                if self.print_broadcast:
                    print(addr)
                    print(data.decode())
        except socket.timeout:
            pass
        
    def get_response_dict(self, data_str):
        property_dict = {}
        server = ''
        for line in data_str.split('\r\n'):
            if line.lower().startswith('server:'):
                server = line[len('server: '):]
            else:
                k = line.split(': ')[0].lower()
                v = line.split(': ')[-1]

                if k != '' and v != '':
                    property_dict[k] = v

        return server, property_dict
    
    """
    - server_name: server name does not have to match exacty, but the entire search str must exist in servers' key
    - servers: dictionary with key: server names, value: property_dict
    - server_idx: if more than one server are found with server_name, then the index is used to pick which server
    
    returns all properties for given server as property dictionary
    """
    def get_property_dict(self, server_name, server_idx=0):
        assert len(self.servers) > 0, 'Error: servers dictionary empty.'
        server_key = [k for k in self.servers.keys() if server_name.lower() in k.lower()]
        assert len(server_key) > 0, 'Error: No server with specified key word found.'
        if len(server_key) > 1:
            print('Multiple servers found for server name (%s). Please use server_idx input argument to specify which server ' \
                  'to use or be more specific when specifying server_name. %s index currently used.'%(server_name, server_idx))              
            print('Servers found:\r\n', ''.join(['\t%s\r\n'%(s) for s in server_key]))
        
        if len(server_key) >= server_idx:
            server_idx = 0
        server_key = server_key[server_idx]

        return self.servers[server_key]
    
    def print_servers(self):
        for server, values in self.servers.items():
            print(server)
            for k, v in values.items():
                print('\t%s: %s'%(k,v))
            print()

class UPNP_Explorer:
    def __init__(self):
        # Broadcast to UPNP Servers & Get Response
        self.SSDP = UPNP_SSDP()
        
    def define_server(self, server_name):
        self.server_name = server_name
        
        # Get location url for specified server
        self.location_url = self.SSDP.get_property_dict(server_name)['location']
        
        self.SCPD = UPNP_SCPD(server_name, self.location_url)
        
    def request_action(self, ctrl_url, service_type, action_name=None, input_args=None):
        if action_name == None:
            action_name = self.SCPD.action_name
            
        if input_args == None:
            input_args = self.SCPD.input_args
            
        self.SOAP = UPNP_SOAP(ctrl_url, service_type, action_name, input_args)
        
    def print_element_pretty(self, obj, tab_cnt=0, filter_txt='', filter_tag='', filter_prt_tag=''):        
        for v in obj.getchildren():
            v_str = '' if v.text == None else v.text.strip()
            t_str = v.tag.split('}')[-1] if '}' in v.tag else v.tag
            if filter_txt.lower() in v_str.lower() and filter_tag.lower() in t_str.lower() and filter_prt_tag.lower() in v.getparent().tag:
                print(''.join(['\t']*tab_cnt)+t_str+'--> '+v_str)

            tab_cnt += 1
            tab_cnt = self.print_element_pretty(v, tab_cnt=tab_cnt, filter_txt=filter_txt, filter_tag=filter_tag)
        tab_cnt -= 1
        
        if tab_cnt < 0:
            return
        return tab_cnt

    def print_xml(self, xml_str):
        print(xml.dom.minidom.parseString(xml_str).toprettyxml())
        
        
class UPNP_SCPD:
    def __init__(self, server_name, location_url):
        self.server_name = server_name
        self.location_url = location_url
        
        # Get SCPD Root XML as ElementTree Element
        response = requests.get(self.location_url)
        self.root = etree.fromstring(response.content, base_url=self.location_url)
        
        # Get service URLs
        self.url_list = self.get_service_urls()
        
    
    def get_service_xml(self, scpd_url):
        response = requests.get(scpd_url)
        self.service_root = etree.fromstring(response.content, base_url=scpd_url)
        
    def get_service_urls(self):
        # Get Base URL
        urlbase = self._scpd_filter(self.root, filter_tag='urlbase')
        if len(urlbase) >= 1:
            self.url_soap_req_base = urlbase[0].text
        else:
            self.url_soap_req_base = ':'.join(self.location_url.split(':')[:-1]) + ':' + self.location_url.split(':')[-1].split('/')[0]

        burl = self.url_soap_req_base
        
        # Get Service URLs    
        ctrl = self._scpd_filter(self.root, filter_prt_tag='service', filter_tag='controlurl')
        scpd = self._scpd_filter(self.root, filter_prt_tag='service', filter_tag='scpdurl')
        stype = self._scpd_filter(self.root, filter_prt_tag='service', filter_tag='servicetype')

        self.url_list = []
        for c, s, st in zip(ctrl, scpd, stype):
            url_dict = dict({'ctrl': urljoin(burl, c.text), 'scpd': urljoin(burl, s.text), 'type': st.text})
            self.url_list += [url_dict]      
        return self.url_list
    
    def _scpd_filter(self, obj, results=None, filter_txt='', filter_tag='', filter_prt_tag='', exact=False):
        if results == None:
            results = []

        for v in obj.getchildren():
            v_str = '' if v.text == None else v.text.strip()
            t_str = v.tag.split('}')[-1] if '}' in v.tag else v.tag
            p_str = v.getparent().tag.split('}')[-1] if '}' in v.getparent().tag else v.getparent().tag
            if exact: # Filter is exact match
                if filter_txt.lower() == v_str.lower() and filter_tag.lower() == t_str.lower() and filter_prt_tag.lower() == p_str.lower():
                    results += [v]
            else: # Filter Contains key word
                if filter_txt.lower() in v_str.lower() and filter_tag.lower() in t_str.lower() and filter_prt_tag.lower() in p_str.lower():
                    results += [v]

            self._scpd_filter(v, results=results, filter_txt=filter_txt, filter_tag=filter_tag, filter_prt_tag=filter_prt_tag, exact=exact)

        return results
    
    """
        - arg_dir: None, 'in', 'out'
    """
    def get_action_arguments(self, action_name, arg_dir=None):
        action = self._scpd_filter(self.service_root, filter_tag='name', filter_prt_tag='action', filter_txt=action_name)
        if len(action) == 0:
            return

        action = action[0].getparent()
        arguments = self._scpd_filter(action, filter_prt_tag='argument', filter_tag='name')
        if arg_dir == None:
            return arguments

        # Get arguments only for specified direction
        filtered_args = []
        directions = self._scpd_filter(action, filter_prt_tag='argument', filter_tag='direction')
        for a, d in zip(arguments, directions):
            if d.text == arg_dir:
                filtered_args += [a]

        return filtered_args
        
    def pretty_print_actions(self, name_filter='', show_datatype=False): 
        actionlist = self._scpd_filter(self.service_root, filter_tag='actionlist')[0]
        actions = self._scpd_filter(actionlist, filter_tag='action')
        for action in actions:
            name = self._scpd_filter(action, filter_tag='name')[0].text
            if name_filter.lower() not in name.lower():
                continue

            arguments = self._scpd_filter(action, filter_tag='argument', filter_prt_tag='argumentlist')

            print_str = name + '('
            in_args, out_args = [], []
            for arg in arguments:
                aname = self._scpd_filter(arg, filter_tag='name', filter_prt_tag='argument')[0].text
                adir = self._scpd_filter(arg, filter_tag='direction', filter_prt_tag='argument')[0].text

                desc_dict = self.get_statevar_desc(name, aname)
                dt = '[%s]'%(desc_dict['dataType']) if 'dataType' in desc_dict and show_datatype else ''

                if adir == 'in':
                    in_args += [aname + dt]
                elif adir == 'out':
                    out_args += [aname + dt]
            print_str += ', '.join(in_args) + ')'
            if len(out_args) > 0:
                print_str += '\r\n\tReturns: (%s)' % (', '.join(out_args)) 
            print(print_str)
            
    def get_relatedstatevariable(self, action_name, arg_name):
        action = self._scpd_filter(self.service_root, filter_prt_tag='action', filter_tag='name', filter_txt=action_name)
        assert len(action) > 0, '(%s) action not found'%(action_name)
        action = action[0].getparent()
        argument = self._scpd_filter(action, filter_prt_tag='argument', filter_tag='name', filter_txt=arg_name)
        assert len(argument) > 0, '(%s) argument not found'%(arg_name)
        argument = argument[0].getparent()
        return self._scpd_filter(argument, filter_tag='relatedstatevariable')[0].text

    def get_statevar_desc(self, action_name, arg_name, show_id=False):
        rsv = self.get_relatedstatevariable(action_name, arg_name)
        statevar = self._scpd_filter(self.service_root, filter_prt_tag='statevariable', filter_tag='name', filter_txt=rsv, exact=True)[0].getparent()

        tags = {}
        for sv_tag in statevar.getchildren():
            tg = sv_tag.tag
            if not(show_id) and 'name' in tg:
                continue

            if '}' in tg:
                tg = tg.split('}')[-1]

            if any([v in tg.lower() for v in ['list', 'range']]):
                arglist = []
                for item in sv_tag.getchildren():
                    arglist += [item.text]
                text = ', '.join(arglist)
            else:
                text = sv_tag.text
            tags[tg] = text

        return tags
    
    def _guess_arg_value(self, desc): 
        datatype = ''
        for k, v in desc.items():
            kl = k.lower()
            if k != '':
                if 'list' in kl and len(v.split(',')) == 1:
                    return v
                elif 'default' in kl:
                    if datatype in ['ui2', 'ui4', 'i2', 'i4']:
                        return int(v)
                    elif datatype in ['string', 'str']:
                        return str(v)
                    else:
                        return v
                elif 'datatype' in kl:
                    datatype = v
                    if v in ['ui2', 'ui4']:
                        return 0  
                elif 'range' in kl and len(v.split(',')) > 1:
                    try:
                        return int(v.split(',')[0])
                    except:
                        pass
        return None

    def define_action(self, action_name):
        self.action_name = action_name
        
        args = self.get_action_arguments(action_name, arg_dir='in')
        assert args != None, 'There was no (%s) action found.'%(action_name)

        arg_dict = {}
        for arg in args:
            desc = self.get_statevar_desc(action_name, arg.text)                    
            arg_dict[arg.text] = self._guess_arg_value(desc)

        self.input_args = arg_dict
        return arg_dict

    def print_action_arg_desc(self, action_name):
        for arg_dir in ['in', 'out']:
            print('_______%sput arguments________'%(arg_dir))
            args = self.get_action_arguments(action_name, arg_dir=arg_dir)
            assert args != None, 'There was no (%s) action found.'%(action_name)

            for arg in args:
                print(arg.text)
                desc = self.get_statevar_desc(action_name, arg.text)
                for k, v in desc.items():
                    if k != '':
                        print('\t%s: %s'%(k, v))
            print()
    
class UPNP_SOAP:
    SOAP_ENVELOPE = 'http://schemas.xmlsoap.org/soap/envelope'
    SOAP_ENCODING = 'http://schemas.xmlsoap.org/soap/encoding/'
    
    def __init__(self, ctrl_url, service_type, action_name, input_args):
        self.ctrl_url = ctrl_url
        self.service_type = service_type
        self.action_name = action_name
        
        if input_args == None:
            input_args = {}
        self.input_args = input_args
        
        self.soap_call()
    
    def create_soap_body(self, encoding='utf-8'):
        soap_env = '{%s}'%self.SOAP_ENVELOPE
        m = '{%s}'%self.service_type
        root = etree.Element(soap_env+'Envelope', nsmap={'SOAP-ENV': self.SOAP_ENVELOPE})
        root.attrib[soap_env+'encodingStyle'] = self.SOAP_ENCODING
        body = etree.SubElement(root, soap_env+'Body')

        action = etree.SubElement(body, m+self.action_name, nsmap={'m': self.service_type})
        for key, value in self.input_args.items():
            etree.SubElement(action, key).text = str(value)

        body = etree.tostring(root, encoding=encoding, xml_declaration=True)

        return body

    def soap_call(self):
        self.soap_body = self.create_soap_body()
        soap_action = "%s#%s"%(self.service_type, self.action_name)

        self.headers = {
            'SOAPAction': '"%s"'%(soap_action),
            'Host': '192.168.1.14:9197',
            'Content-Type': 'text/xml',
            'Content-Length': str(len(self.soap_body)),
        }

        response = requests.post(self.ctrl_url, self.soap_body, headers=self.headers)
        
        self.response = response.content.strip()
        return self.response
        

### STEP 1: Broadcast

In [141]:
upnp = UPNP_Explorer() # Broadcast SSDP to UPNP Servers and Get Replies

In [142]:
upnp.SSDP.print_servers()

Samsung-Linux/4.1, UPnP/1.0, Samsung_UPnP_SDK/1.0_0
	http/1.1 200 ok: HTTP/1.1 200 OK
	cache-control: max-age=1800
	date: Wed, 15 Jul 2020 15:54:29 GMT
	location: http://192.168.1.14:9197/dmr
	st: upnp:rootdevice
	usn: uuid:036cb031-f5f8-4831-bcd7-c0259ee25740::upnp:rootdevice
	content-length: 0
	bootid.upnp.org: 4

Samsung-Linux/4.1, UPnP/1.0, Samsung_UPnP_SDK/1.0_1
	http/1.1 200 ok: HTTP/1.1 200 OK
	cache-control: max-age=1800
	date: Wed, 15 Jul 2020 15:54:29 GMT
	location: http://192.168.1.14:7678/nservice/
	st: upnp:rootdevice
	usn: uuid:f4ec0a2d-c0aa-4f9d-904a-ecc457202ba9::upnp:rootdevice
	content-length: 0
	bootid.upnp.org: 5

Linux/2.6.35.14-grsec2.2.2, UPnP/1.0, Portable SDK for UPnP devices/1.6.13_0
	http/1.1 200 ok: HTTP/1.1 200 OK
	cache-control: max-age=1800
	date: Wed, 15 Jul 2020 15:54:30 GMT
	ext:: EXT:
	location: http://192.168.1.3:9080
	opt: "http://schemas.upnp.org/upnp/1/0/"; ns=01
	01-nls: b008ee90-1dd1-11b2-ba25-b1c1c18d1437
	x-user-agent: NRDP MDX
	x-friendly-nam

# Parse SCPD(Service Control Point Definition) XMLs

### STEP 2: Define Server
- Gets Root XML using Location URL

In [143]:
SEARCH_SERVER_NAME = 'samsung'
url_list = upnp.define_server(SEARCH_SERVER_NAME)

print('Location URL: ', upnp.SCPD.location_url) # Contains Device Definition and Service List

Multiple servers found for server name (samsung). Please use server_idx input argument to specify which server to use or be more specific when specifying server_name. 0 index currently used.
Servers found:
 	Samsung-Linux/4.1, UPnP/1.0, Samsung_UPnP_SDK/1.0_0
	Samsung-Linux/4.1, UPnP/1.0, Samsung_UPnP_SDK/1.0_1

Location URL:  http://192.168.1.14:9197/dmr


In [144]:
# Pretty Print SCPD Root XML
upnp.print_element_pretty(upnp.SCPD.root)

specVersion--> 
	major--> 1
	minor--> 0
device--> 
	deviceType--> urn:schemas-upnp-org:device:MediaRenderer:1
	X_compatibleId--> MS_DigitalMediaDeviceClass_DMR_V001
	X_deviceCategory--> Display.TV.LCD Multimedia.DMR
	X_DLNADOC--> DMR-1.50
	friendlyName--> [TV] Samsung 6 Series (75)
	manufacturer--> Samsung Electronics
	manufacturerURL--> http://www.samsung.com/sec
	modelDescription--> Samsung TV DMR
	modelName--> UN75NU6950
	modelNumber--> AllShare1.0
	modelURL--> http://www.samsung.com/sec
	serialNumber--> 07ZX3CDN101794W
	UDN--> uuid:036cb031-f5f8-4831-bcd7-c0259ee25740
	iconList--> 
		icon--> 
			mimetype--> image/jpeg
			width--> 48
			height--> 48
			depth--> 24
			url--> /icon_SML.jpg
		icon--> 
			mimetype--> image/jpeg
			width--> 120
			height--> 120
			depth--> 24
			url--> /icon_LRG.jpg
		icon--> 
			mimetype--> image/png
			width--> 48
			height--> 48
			depth--> 24
			url--> /icon_SML.png
		icon--> 
			mimetype--> image/png
			width--> 120
			height--> 120
			depth--> 24
	

### STEP 3a: Choose URL Set
- Choose Control URL (SOAP Commands sent to this URL)
- Choose SCPD URL (Service URL with Argument & Action Definition XML)
- Choose Type (Define for SOAP Requests)

In [145]:
for i, urls in enumerate(upnp.SCPD.url_list):
    print('---- URL Set Index %s ----'%(i))
    for k, v in urls.items():
        print('%s: %s'%(k,v))
    print()

---- URL Set Index 0 ----
ctrl: http://192.168.1.14:9197/upnp/control/RenderingControl1
scpd: http://192.168.1.14:9197/RenderingControl_1.xml
type: urn:schemas-upnp-org:service:RenderingControl:1

---- URL Set Index 1 ----
ctrl: http://192.168.1.14:9197/upnp/control/ConnectionManager1
scpd: http://192.168.1.14:9197/ConnectionManager_1.xml
type: urn:schemas-upnp-org:service:ConnectionManager:1

---- URL Set Index 2 ----
ctrl: http://192.168.1.14:9197/upnp/control/AVTransport1
scpd: http://192.168.1.14:9197/AVTransport_1.xml
type: urn:schemas-upnp-org:service:AVTransport:1



In [146]:
# Choose Index from above list
USE_IDX = 0

CTRL_URL = upnp.SCPD.url_list[USE_IDX]['ctrl']
SCPD_URL = upnp.SCPD.url_list[USE_IDX]['scpd']
SERVICE_TYPE = upnp.SCPD.url_list[USE_IDX]['type']
print(CTRL_URL)
print(SCPD_URL)
print(SERVICE_TYPE)

http://192.168.1.14:9197/upnp/control/RenderingControl1
http://192.168.1.14:9197/RenderingControl_1.xml
urn:schemas-upnp-org:service:RenderingControl:1


### STEP 3b: Get Service XML
- Gets the Service XML using the SCPD_URL
- Service XML contains Action & Argument Definitions

In [147]:
upnp.SCPD.get_service_xml(SCPD_URL)

# Print Service XML
upnp.print_element_pretty(upnp.SCPD.service_root)

specVersion--> 
	major--> 1
	minor--> 0
actionList--> 
	action--> 
		name--> ListPresets
		argumentList--> 
			argument--> 
				name--> InstanceID
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_InstanceID
			argument--> 
				name--> CurrentPresetNameList
				direction--> out
				relatedStateVariable--> PresetNameList
	action--> 
		name--> SelectPreset
		argumentList--> 
			argument--> 
				name--> InstanceID
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_InstanceID
			argument--> 
				name--> PresetName
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_PresetName
	action--> 
		name--> GetMute
		argumentList--> 
			argument--> 
				name--> InstanceID
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_InstanceID
			argument--> 
				name--> Channel
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_Channel
			argument--> 
				name--> CurrentMute
				direction--> out
				relatedStateVariable--> Mute
	action--> 
		name--> SetMute
		argumentLi

# SOAP Preparation

### Step 4a: Choose an Action For SOAP Request

In [148]:
upnp.SCPD.pretty_print_actions()

ListPresets(InstanceID)
	Returns: (CurrentPresetNameList)
SelectPreset(InstanceID, PresetName)
GetMute(InstanceID, Channel)
	Returns: (CurrentMute)
SetMute(InstanceID, Channel, DesiredMute)
GetVolume(InstanceID, Channel)
	Returns: (CurrentVolume)
SetVolume(InstanceID, Channel, DesiredVolume)
X_GetAspectRatio(InstanceID)
	Returns: (AspectRatio)
X_SetAspectRatio(InstanceID, AspectRatio)
X_Move360View(InstanceID, LatitudeOffset, LongitudeOffset)
X_Zoom360View(InstanceID, ScaleFactorOffset)
X_Origin360View(InstanceID)
X_ControlCaption(InstanceID, Operation, Name, ResourceURI, CaptionURI, CaptionType, Language, Encoding)
X_GetCaptionState(InstanceID)
	Returns: (Captions, EnabledCaptions)
X_GetServiceCapabilities(InstanceID)
	Returns: (ServiceCapabilities)
X_SetZoom(InstanceID, x, y, w, h)
X_GetTVSlideShow(InstanceID)
	Returns: (CurrentShowState, CurrentThemeId, TotalThemeNumber)
X_SetTVSlideShow(InstanceID, CurrentShowState, CurrentShowTheme)


In [149]:
ACTION_NAME = 'SetMute'

In [150]:
upnp.SCPD.print_action_arg_desc(ACTION_NAME)

_______input arguments________
InstanceID
	dataType: ui4
Channel
	dataType: string
	allowedValueList: Master
DesiredMute
	dataType: boolean

_______output arguments________



### Step 4b: Define Action used and Action's Input Arguments

In [151]:
argument_inputs = upnp.SCPD.define_action(ACTION_NAME)

print(upnp.SCPD.input_args)

{'InstanceID': 0, 'Channel': 'Master', 'DesiredMute': None}


### Step 4c: Change any input action values if necessary or desired

In [152]:
# Modify any arguments you would like to manually change
argument_inputs['DesiredMute'] = 1

print(upnp.SCPD.input_args)

{'InstanceID': 0, 'Channel': 'Master', 'DesiredMute': 1}


### Step 5: SOAP Call

In [153]:
upnp.request_action(CTRL_URL, SERVICE_TYPE) # ACTION_NAME & INPUT_ARGS defined from SCPD.define_action

# OR if ACTION_NAME & INPUT_ARGS are defined elsewhere

# upnp.request_action(CTRL_URL, SERVICE_TYPE, ACTION_NAME, INPUT_ARGS)

In [154]:
upnp.print_xml(upnp.SOAP.soap_body)

<?xml version="1.0" ?>
<SOAP-ENV:Envelope SOAP-ENV:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/" xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope">
	<SOAP-ENV:Body>
		<m:SetMute xmlns:m="urn:schemas-upnp-org:service:RenderingControl:1">
			<InstanceID>0</InstanceID>
			<Channel>Master</Channel>
			<DesiredMute>1</DesiredMute>
		</m:SetMute>
	</SOAP-ENV:Body>
</SOAP-ENV:Envelope>



In [155]:
upnp.print_xml(upnp.SOAP.response)

<?xml version="1.0" ?>
<s:Envelope s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/" xmlns:s="http://schemas.xmlsoap.org/soap/envelope/">
	<s:Body>
		<u:SetMuteResponse xmlns:u="urn:schemas-upnp-org:service:RenderingControl:1"/>
	</s:Body>
</s:Envelope>

